# 📘 Documentação da Camada Bronze – Notebook de Ingestão

Este notebook realiza a **extração de dados do banco PostgreSQL** hospedado no Azure e grava as tabelas na **camada Bronze** do Data Lake em formato Delta. Abaixo estão as ações executadas:

---

## 🔹 Configuração da Conexão
- Definição da **URL JDBC** para conexão com o banco PostgreSQL (`jdbc:postgresql://.../vendas?sslmode=require`).  
- Criação de um dicionário `connection_properties` contendo:
  - Usuário (`user`)  
  - Senha (`password`)  
  - Driver (`org.postgresql.Driver`)  

---

## 🔹 Extração de Dados (Leitura via Spark)
Foram extraídas as seguintes tabelas do schema `public` no PostgreSQL:

- **Clientes** → `df_clientes`  
- **Produtos** → `df_produtos`  
- **Pedidos** → `df_pedido`  
- **Itens_Pedido** → `df_itens_pedido`  

Cada DataFrame foi exibido com `display()` para validação.

---

## 🔹 Persistência na Camada Bronze
Os DataFrames foram gravados em formato **Delta** na camada Bronze, utilizando `.write.format("delta").mode("append").saveAsTable(...)`.

- `df_clientes` → `bronze.vendas_clientes`  
- `df_produtos` → `bronze.vendas_produtos`  
- `df_pedido` → `bronze.vendas_pedidos`  
- `df_itens_pedido` → `bronze.vendas_itens__pedidos`  

---

## 🎯 Objetivo da Camada Bronze
- Armazenar os dados **brutos** vindos do sistema transacional (PostgreSQL).  
- Garantir **persistência confiável** em formato Delta para futuras transformações.  
- Servir como base para a construção das camadas **Silver** e **Gold** dentro da arquitetura **Medallion**.  


In [0]:
jdbc_url = (
    "jdbc:postgresql://azuremaniezzijp.postgres.database.azure.com:5432/vendas"
    "?sslmode=require"
)

# OBSERVAÇÃO : NA URL SEMPRE LEMBRE DE COLOCAR O BANCO DE DADOS NO FINAL DA URL

connection_properties = {
    "user": "PgAdminJp",
    "password": "***",
    "driver": "org.postgresql.Driver"
}




In [0]:

# Extraindo tabela Clientes do banco postgres


df_clientes = spark.read.jdbc(
    url=jdbc_url,
    table="public.clientes",
    properties=connection_properties
)

display(df_clientes)


# Extraindo tabela Produtos do banco postgres

df_produtos = spark.read.jdbc(
    url=jdbc_url,
    table="public.produtos",
    properties=connection_properties
)

display(df_produtos)



# Extraindo tabela Pedidos do banco postgres

df_pedido = spark.read.jdbc(
    url=jdbc_url,
    table="public.pedido",
    properties=connection_properties
)


display(df_pedido)

# Extraindo tabela Itens_Pedidos do banco postgres

df_itens_pedido = spark.read.jdbc(
    url=jdbc_url,
    table="public.itens_pedido",
    properties=connection_properties
)

display(df_itens_pedido)

df_clientes.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("bronze.vendas_clientes")


df_produtos.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("bronze.vendas_produtos")


df_pedido.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("bronze.vendas_pedidos")


df_itens_pedido.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("bronze.vendas_itens__pedidos")

